In [2]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split


# For pandas , os ...
import os
import math
import pandas as pd
import numpy as np


# For Progress Bar
from tqdm import tqdm


### Dataset

In [66]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(train_set, valid_set, seed):
    '''Split provided training data into training set and validation set'''
    #valid_set_size = int(valid_ratio * len(data_set)) 
    #train_set_size = len(data_set) - valid_set_size
    #train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    
    print('the size of train set' , np.array(train_set))
    return np.array(train_set), np.array(valid_set)


def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds


def select_feat(train_data, valid_data , select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,0], valid_data[:,0]
    raw_x_train, raw_x_valid = train_data[:,1:], valid_data[:,1:]

    #if select_all:
    #    feat_idx = list(range(raw_x_train.shape[1]))
    #else:
    #    feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        
    return raw_x_train, raw_x_valid, y_train, y_valid


class IncompleteDataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = y
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        #if self.y is None:
            #return self.x[idx]
        #else:
            #return self.x[idx], self.y[idx]
            
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], int(self.y[idx])

    def __len__(self):
        return len(self.x)



    

### Model

In [3]:
class incomplete_cls(nn.Module):
    def __init__(self):
        super(incomplete_cls, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(8, 2)  # 2 classes ,0 for complete and 1 for incomplete
        )
        
        self.dropout = nn.Dropout(0.1) # 10% probability of dropout
        
        

    def forward(self, x):
        
        x = self.layers(x)
        #print('x shape is' ,x.shape)
        out = x.squeeze(1) # (B,1)
        #print('out shape is' , out.shape)
        return out

In [68]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.CrossEntropyLoss() # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    #optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./incomplete_models'):
        os.mkdir('./incomplete_models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []
        train_accs = []
        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            #print('x shape in moodel is ',x.shape)
            pred = model(x)         
            #print( 'y shape is ',y.shape)
            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically    
            loss = criterion(pred, y) # pred Tensor[a,b,c] , y :int
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            # Compute the accuracy for current batch.
            acc = (pred.argmax(dim=-1) == y.to(device)).float().mean()
            train_accs.append(acc)
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
            


        mean_train_loss = sum(loss_record)/len(loss_record)
        train_acc = sum(train_accs) / len(train_accs)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] , acc = {train_acc:.5f}")
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return


In [71]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 6666666,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 16, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './incomplete_models/incomplete_models.ckpt'  # Your model will be saved here.
}

# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: n x 9 (  label + (x,y) * 4 = 9 ) 

train_data = pd.read_csv('./10%train_incomplete.csv').values
valid_data = pd.read_csv('./10%val_incomplete.csv').values
train_data, valid_data = train_valid_split(train_data, valid_data, config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} """)

# Select features
x_train, x_valid, y_train, y_valid = select_feat(train_data, valid_data,  config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = IncompleteDataset(x_train, y_train), \
                                            IncompleteDataset(x_valid, y_valid), \
                                            IncompleteDataset(x_valid, None)
                                            
# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)


model = incomplete_cls().to(device) # put your model and data on the same computation device
print(model)




the size of train set [[1.         0.         0.09567901 ... 0.15020576 0.         0.15020576]
 [0.         0.07908951 0.08796296 ... 0.15534979 0.07716049 0.15072016]
 [0.         0.19251543 0.09567901 ... 0.15329218 0.19251543 0.15329218]
 ...
 [0.         0.54899691 0.77880658 ... 0.83230453 0.54899691 0.83230453]
 [0.         0.77893519 0.78703704 ... 0.83539095 0.77893519 0.83539095]
 [1.         0.9382716  0.78806584 ... 0.83847737 0.9382716  0.83847737]]
train_data size: (1144, 9) 
valid_data size: (297, 9) 
number of features: 8
incomplete_cls(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)


NameError: name 'writer' is not defined

In [72]:

# Train your model
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|██████████| 72/72 [00:00<00:00, 701.34it/s, loss=0.636]


Epoch [1/5000]: Train loss: 0.6755, Valid loss: 0.6403
[ Train | 001/5000 ] , acc = 0.68663
Saving model with loss 0.640...


Epoch [2/5000]: 100%|██████████| 72/72 [00:00<00:00, 769.80it/s, loss=0.576]


Epoch [2/5000]: Train loss: 0.6052, Valid loss: 0.5702
[ Train | 002/5000 ] , acc = 0.88542
Saving model with loss 0.570...


Epoch [3/5000]: 100%|██████████| 72/72 [00:00<00:00, 739.62it/s, loss=0.399]


...

Epoch [4304/5000]: Train loss: 0.0602, Valid loss: 0.0438
[ Train | 4304/5000 ] , acc = 0.98264


Epoch [4305/5000]: 100%|██████████| 72/72 [00:00<00:00, 940.05it/s, loss=0.00446]


Epoch [4305/5000]: Train loss: 0.0744, Valid loss: 0.0556
[ Train | 4305/5000 ] , acc = 0.97569


Epoch [4306/5000]: 100%|██████████| 72/72 [00:00<00:00, 772.05it/s, loss=0.0167]


Epoch [4306/5000]: Train loss: 0.0658, Valid loss: 0.0616
[ Train | 4306/5000 ] , acc = 0.98090

Model is not improving, so we halt the training session.


### Predict the testing set

In [73]:



test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

model = incomplete_cls().to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 

100%|██████████| 19/19 [00:00<00:00, 9996.46it/s]


### Print the Results

In [5]:
device = 'cuda:0'
model = incomplete_cls().to(device)

config = {
    'seed': 6666666,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 16, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './incomplete_models/incomplete_models.ckpt'  # Your model will be saved here.
}
model.load_state_dict(torch.load(config['save_path']))

<All keys matched successfully>

In [83]:
small_test_set = pd.read_csv('./incomplete_data/test.csv')
test_for_incomplete = np.array(small_test_set)
print(test_for_incomplete)

[[0.         0.22839506 0.02353395 0.22839506 0.02353395 0.27726337
  0.         0.27726337]
 [0.         0.87294239 0.05362654 0.87294239 0.05362654 0.92901235
  0.         0.92901235]
 [0.95871914 0.12654321 1.         0.12654321 1.         0.17438272
  0.95871914 0.17438272]
 [0.07330247 0.24434156 0.1554784  0.24434156 0.1554784  0.29372428
  0.07330247 0.29372   ]]


In [84]:


test_dataset = IncompleteDataset(test_for_incomplete ,None)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
preds = predict(test_loader, model, device) 


100%|██████████| 1/1 [00:00<00:00, 60.77it/s]


In [27]:

x = torch.FloatTensor([[[0.7741, 0.3802],
         [0.7742, 0.3280],
         [0.7037, 0.3278],
         [0.7036, 0.3800]],
        [[0.9912, 0.9823],
         [0.9999, 1.1111],
         [0.7037, 0.3278],
         [0.7036, 0.3800]],
        [[ -0.000733782 ,0.857666 ],
         [ 0.0197159 ,0.857738],
         [ 0.019791 ,0.819874] ,
         [-0.000658668 ,0.819802]]]).to(device)



# original shape is (1,4,2)
# Need to reshpaed to (1,8)
x = x.reshape(-1,8)
print(x.shape)
pred = model(x)

torch.Size([3, 8])


In [28]:

print(pred)

tensor([[ 2.9022, -3.0275],
        [-0.6903,  0.5122],
        [-3.4393,  3.1640]], device='cuda:0', grad_fn=<SqueezeBackward1>)
